# NLP - Assignment 2

## Imports

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from bertopic import BERTopic
import numpy as np

## Preprocess Data

In [ ]:
path = "./us_equities_news_dataset.csv"

# Load the news dataset
news_dataset = pd.read_csv("./us_equities_news_dataset.csv")
news_dataset.head()

In [7]:
# Replace with different versions of the preprocessed dataset
train_data_v1 = None 
train_data_v2 = None 
train_data_v3 = None 

## Train Topic Models

In [6]:
def train_topic_model(train_data, model_type='LDA', num_topics=10):
    """
    Train a topic model on the given training data.

    Parameters:
    - train_data: list of str, the text to train the model on
    - model_type: str, the type of model to train ('LDA', 'FLSA-W', 'BERTopic')
    - num_topics: int, the number of topics to generate

    Returns:
    - model: the trained model
    - topics: the topics generated by the model
    """
    if model_type == 'LDA':
        # Replace with actual LDA training code
        lda = None
        topics = None
        
        return lda, topics
    
    elif model_type == 'FLSA-W':
        # Replace with actual FLSA-W training code
        flsa_w_model = None
        topics = None
        
        return flsa_w_model, topics
    
    elif model_type == 'BERTopic':
        # Train BERTopic model
        topic_model = BERTopic(nr_topics=num_topics)
        topics, _ = topic_model.fit_transform(train_data)
        
        return topic_model, topics
    
    else:
        raise ValueError("Invalid model_type. Choose from 'LDA', 'FLSA-W', 'BERTopic'.")

### LDA

### FLSA-W

### BERTopic

In [ ]:
# Train BERTopic model on the first version of the preprocessed dataset
bertopic_model, bertopic_topics = train_topic_model(train_data_v1, model_type='BERTopic', num_topics=10)

## Iteration 1

## Iteration 2

## Iteration 3

## Iteration ....

## Evaluation

### Qualitative

### Quantitative

## Output of Final Topic Model